# Floyd-Warshall Algorithm

Probably the most common algorithm for finding shortest path is the Dijkstra algorithm. Dijkstra algorithm can find the shorstest path from one origin node to all reachable destinations. However, oftentimes we want to know the shortest path from any origin to any destination. While it is possible to run Dijkstra algorithm for all possible origins, Floyd-Warshall algorithm is a simple choice for computing all-to-all shortest paths. This notebook concerns on how to implement the Floyd-Warshal algorithm. 

Interestingly, even though the algorithm is name after two independet researchers, Robert Floyd and Stephen Warshall, there were several work around 1960 related to the same problem. Robert Floyd published in 1962 an algorithm for finding all-to-shortest paths. In the same year, Stephen Warshall published a paper describing essentially the same algorithm in order to find the Transitive Closure of a Graph. Transitive Closure refers to determining for all node pairs if there is at least one path connecting them. These two problems are similar as finding a shortest path means that there is a path. Few years later, Bernard Roy published essentially the same algorithm in French and apparently remained unnoticed. Nevertheless, sometimes the same algorithm is often referred to as Roy-Floyd-Warshall algorithm. Still in 1962, Peter Ingerman published the version of the algorithm that is currently used.

It is assumed to a directed graph $G$ containing several nodes $n$ in the node set $N$ arcs [u,v] in the link (arc) $A$ where $u$ refers to the upstream node of the link and $v$ refers to the downstream node. Also, each arc [u,v] has its associated cost where in the domain of transportation networks is often associated to travel time along the link.

Let's firt have a graph as example. The graph below has $N=[1,2,3,4,5]$ and arcs $ A = [(1,2),(2,1),(1,3), (3,1), (2,4),(4,2),(4,3),(3,5), (5,4)]$  with their associated costs. Except from 3-5 to 4 there are multiple paths from node to node. So let's find all the shortest path in this graph.

<img src="graphexample.png">

# Dependencies

In [1]:
import numpy
import networkx

The first step let's translate the graph we see in the picture into a data structure for graphs:

In [3]:
nodes = [0,1,2,3,4]

arcs_costs = [(0,1, 1.5),(1,0, 2.0),(0,2, 4.0), (2,0, 1.0), (1,3, 1.5),(3,1, 1.5),(3,2, 0.5),(2,4, 1.5),(4,3, 2.0)]

arcs = []
costs = {}

for u,v,cost in arcs_costs:
    arcs.append((u,v))
    costs[u,v] = cost


So now we have in appropriate variables the list of nodes, the links described as the nodes they connect and the link costs. The basic idea of Floyd algorithm is very simple. Whever we have three nodes, $u$, $v$, $z$ in which may be for example for our example nodes 4,2,and 1 and for now we did not know that there is a path connecting 4 to 1. However, we know the cost from link 4 to 2 (1.5) and the path (link (4,2)); similarly from 2 to 1 with cost 2.0. Therefore, we can travel from 4 to 1 by first taking the path that connects 4-2 and after taking the path that connects 2 to 1. Logical, right? So let's not get confused by the following relationship:

$$t_{uz} \leq t_{uv} + t_{vz}$$

which summarize the idea. If we want to travel between node $u$ and $z$ (4 to 1) and we have a cost which is higher than the sum of $t_{uv}$ (4-2) and $t_{vz}$ (2-1) it can be the shortest path as we can take the two sub paths (4-2) and (2-1) instead. Floyd algorithm works by sistematically looking if there is a node $v$ that can can connect any node $u$ and $z$ and labelling it appropriatelly when it does. 

$$ if t_{uz} \geq t_{uv} + t_{vz} \implies t_{uz} = t_{uv} + t_{vz}$$

This operation is a performed testing all possible combinations of $u$, $v$ and $z$. Here is also presented a version in which we store the paths as well with a matrix that stores the first node to be visited for any path $u$, $v$. So we have two square matrix with dimensions equal to the number of nodes called times, and next. The matrix times is initialed with all values to infinity (meaning we do not know the path fro $u$ to $z$ so it is infinite). The matrix next starts with its value $None$ or null as we do not now the next link for any path. Then, we can update the values for each arc in the graph. The code below starts the matrices and perform the initial update based on the links in the graphs.  

In [11]:
n = len(nodes)

times = numpy.empty((n,n))
next_matrix = numpy.empty((n,n), numpy.int)
times[:] = numpy.inf
next_matrix[:] = -1

for u,v in arcs:
    times[u, v] = costs[u, v]
    next_matrix[u,v] = v

print('times after initialization:')
print(times)

print('matrix of next link after initialization:')

print(next_matrix)

times after initialization:
[[ inf  1.5  4.   inf  inf]
 [ 2.   inf  inf  1.5  inf]
 [ 1.   inf  inf  inf  1.5]
 [ inf  1.5  0.5  inf  inf]
 [ inf  inf  inf  2.   inf]]
matrix of next link after initialization:
[[-1  1  2 -1 -1]
 [ 0 -1 -1  3 -1]
 [ 0 -1 -1 -1  4]
 [-1  1  2 -1 -1]
 [-1 -1 -1  3 -1]]


Now we systematically look for all combinations of three nodes u,v,z and check whether v is a sub-path from u to z. This is done in a 3 nested for-loops.

In [12]:
for u in range(n):
    for v in range(n):
        for z in range(n):
            if times[v,z] > times[v,u]+times[u,z]:
                times[v,z] = times[v,u]+times[u,z]
                next_matrix[v,z] = next_matrix[v,u]

print("shortest path times are:")
print(times)

print("next matrix:")
print(next_matrix)

shortest path times are:
[[ 3.5  1.5  3.5  3.   5. ]
 [ 2.   3.   2.   1.5  3.5]
 [ 1.   2.5  4.   3.5  1.5]
 [ 1.5  1.5  0.5  3.   2. ]
 [ 3.5  3.5  2.5  2.   4. ]]
next matrix:
[[1 1 1 1 1]
 [0 3 3 3 3]
 [0 0 4 4 4]
 [2 1 2 1 2]
 [3 3 3 3 3]]


So the travel times between each node pair are computed. Observe that the diagonal is the travel time from link $u$ back to link $u$. This value could have been zero if we have started the times matrix with the value zero. Nevertheless, the value on that matrix is the time shortest path leaving a node and returning to that same node.

We have computed the times and the next matrix. The next matrix encodes the path, but we need to further looking to it to obtain a complete path. The element [u,v] of that matrix gives the first node to be visited when traveling from $u$ to $v$. Let's say the first node is $z$. From node $z$ we look to the same matrix to find the first node to be visited from node $z$ to node $v$. We can do that until reach the node $v$. The following code obtains the sequence of nodes in the shortest path between nodes $u$ and $v$.

In [15]:
def compute_path(u,v, next_matrix):
    path = []
    
    next_node = u
    
    while next_node != v:
        path.append(next_node)
        next_node = next_matrix[next_node,v]
    
    path.append(v)
    return path

path =compute_path(0,2, next_matrix)
print("node sequence from 0 to 2 is:", path)
path_reverse = compute_path(2,0, next_matrix)
print("the reverse path - from 2 to 0 - is:", path_reverse)

node sequence from 0 to 2 is: [0, 1, 3, 2]
the reverse path - from 2 to 0 - is: [2, 0]


Floyd-Warshall is as simple as that! It is very simple and useful. Nonetheless, when we have a very large number of nodes in a practical application it may be slower than Dijkstra Algorithm well implemented with Heaps. In transportation networks where usually the number of nodes and links have the same order of magnitude, Dijkstra is faster. Keep in mind, however, it usually becomes critical when a graph contains thousands of nodes and links. 

The libraries for graph theory usually has a good implementation of Floyd Algorithm and that is the case of networkx. In the piece of code below, the same network is created using networkx and the same calls are performed similarly to the code above.

In [25]:
graph = networkx.DiGraph()

for node in nodes:
    graph.add_node(node)

for u,v in arcs:
    graph.add_edge(u,v)
    graph.edges[u,v]['weight'] = costs[u,v]

predecessor, times_dic = networkx.floyd_warshall_predecessor_and_distance(graph)

translation = numpy.zeros((n,n))
for node_from in times_dic:
    for node_to in times_dic[node_from]:
        translation[node_from, node_to] = times_dic[node_from][node_to]

print(translation)

print(predecessor[2][3])

[[ 0.   1.5  3.5  3.   5. ]
 [ 2.   0.   2.   1.5  3.5]
 [ 1.   2.5  0.   3.5  1.5]
 [ 1.5  1.5  0.5  0.   2. ]
 [ 3.5  3.5  2.5  2.   0. ]]
4


Observe network returns a nested dictionary, but similar: predecessor[u][v] returns the next link from $u$ to $v$. Similarly, times_dic[u][v] provides the travel time from $u$ to $v$. Also, networkw already assumes that times[u,u] = 0 and therefore the main diagonal is zero.